In [1]:
!pip install requests

In [2]:
!pip install beautifulsoup4

In [3]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 17.0 MB/s eta 0:00:00


# 라이브러리

In [12]:
# 파이썬 표준 라이브러리
import os
import requests
import re
import random
import time
from pprint import pprint
from concurrent import futures

# 파이썬 서드파티 라이브러리
import bs4
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

# 전역 변수

In [5]:
# cpu 갯수
workers = os.cpu_count()
print(workers)

2


In [6]:
news_websites_dict = {
                    'Investing': 'https://kr.investing.com/news/cryptocurrency-news',
                    'Hankyung': 'https://www.hankyung.com/koreamarket/news/crypto',
                    'Bloomingbit': 'https://bloomingbit.io/feed',
                    'Coinreaders': 'https://www.coinreaders.com/',
                    'Cryptonews': 'https://cryptonews.com/kr/news/',
                      }

In [9]:
# Chrome 옵션 설정
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# User-Agent 변경을 위한 옵션 설정
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
headers = {'User-Agent': user_agent}
options.add_argument(f"user-agent={user_agent}")

# 함수

In [27]:
def investing(news_url: bs4.element.Tag) -> dict[str, str, None]:
    """뉴스 URL을 바탕으로 크롤링을 하는 함수

    Args:
        news_url: 뉴스 URL

    Returns:
        {
            "news_title": 뉴스 제목, str
            "news_first_upload_time": 뉴스 최초 업로드 시각, str | None
            "newsfinal_upload_time": 뉴스 최종 수정 시각, str
            "author": 뉴스 작성자
            "news_content": 뉴스 본문, str
            "news_url": 뉴스 URL, str
            "news_website": 뉴스 웹사이트, str
        }
    """
    info = {} # 뉴스 데이터 정보 Dictionary

    # news_url를 찾아서 requests로 HTML GET을 한 다음, BeautifulSoup로 parser
    url = news_url.find("a")["href"]
    html = requests.get(url).text
    time.sleep(0.1)
    soup = BeautifulSoup(html, 'html.parser')

    # 1. 뉴스 데이터의 제목
    title = soup.find('h1', id='articleTitle').text

    # 2. 뉴스 데이터의 최초 업로드 시각과 최종 수정 시각
    first_upload_time = None
    div = soup.find_all('div', {'class': 'flex flex-row items-center'})
    span = div[1].find('span')
    last_upload_time_list = re.split(r'\s+\r\n\s+', span.text)[1].split()
    y_m_d = '-'.join(times[:-1] for times in last_upload_time_list[:3])
    if last_upload_time_list[3] == '오전':
        ap = 'AM'
    else:
        ap = 'PM'
    last_upload_time = y_m_d + ' ' + ap + ' ' + last_upload_time_list[4]

    # 3. 뉴스 데이터의 기사 작성자
    author = None

    # 4. 뉴스 데이터의 본문
    div = soup.find('div', id='article')
    news_content = div.find_all('p')

    # 뉴스 데이터 본문의 데이터 전처리1
    if re.search('(읽기|provided|[a-z0-9]@[a-z0-9])', news_content[-1].text, flags=re.IGNORECASE):
        del news_content[-1]
    # 뉴스 데이터 본문의 데이터 전처리2
    text = ''.join(t.text for t in news_content).strip(r' \t\n\r\f\v')
    text = re.sub(r'([가-힣])\.(\w)', r'\g<1>. \g<2>', text)
    text = re.sub('(\xa0)+', ' ', text)

    info['news_title'] = title
    info['news_first_upload_time'] = first_upload_time
    info['news_last_upload_time'] = last_upload_time
    info['author'] = author
    info['news_content'] = text
    info['news_url'] = url
    info['news_website'] = 'Investing'

    return info

In [ ]:
def crawling(website: str):
    """Crawling을 하는 함수

    Args:
        website: 웹사이트 이름, str

    Returns:
        pass
    """

    assert website in news_websites_dict, f'{website} is not target website.'

    driver = webdriver.Chrome(options=options)

    match website:
        case 'Investing':
            web_page = 'https://kr.investing.com/news/cryptocurrency-news'
            page_num = 1

            for _ in range(2000):
                html = requests.get(web_page).text
                soup = BeautifulSoup(html, 'html.parser')
                news_list = soup.find_all('article', {"data-test": "article-item"})

                for news_url in news_list:
                    info = {}
                    url = news_url.find("a")["href"]
                    html = requests.get(url).text
                    soup = BeautifulSoup(html, 'html.parser')

                    title = soup.find('h1', id='articleTitle').text

                    first_upload_time = None
                    div = soup.find_all('div', {'class': 'flex flex-row items-center'})
                    span = div[1].find('span')
                    last_upload_time_list = re.split(r'\s+\r\n\s+', span.text)[1].split()
                    y_m_d = '-'.join(times[:-1] for times in last_upload_time_list[:3])
                    if last_upload_time_list[3] == '오전':
                        ap = 'AM'
                    else:
                        ap = 'PM'
                    last_upload_time = y_m_d + ' ' + ap + ' ' + last_upload_time_list[4]

                    author = None

                    div = soup.find('div', id='article')
                    news_content = div.find_all('p')

                    info['news_title'] = title
                    info['news_first_upload_time'] = first_upload_time
                    info['news_last_upload_time'] = last_upload_time
                    info['author'] = author

                    if re.search('(읽기|provided|[a-z0-9]@[a-z0-9])', news_content[-1].text, flags=re.re.IGNORECASE):
                        del news_content[-1]


                    text = ''.join(t.text for t in news_content).strip(r' \t\n\r\f\v')
                    text = re.sub(r'([가-힣])\.(\w)', r'\g<1>. \g<2>', text)
                    text = re.sub('(\xa0)+', ' ', text)
                    info['news_content'] = text

                    info['news_url'] = url
                    info['news_website'] = website

                    break

                    page_num += 1
                    web_page = f'https://kr.investing.com/news/cryptocurrency-news/{page_num}'

                time.sleep(random.uniform(0.5, 1.75))

        case 'Hankyung':

        case 'Bloomingbit':

        case 'Coinreaders':

        case 'Cryptonews':


IndentationError: expected an indented block after 'case' statement on line 25 (1482604058.py, line 28)

# Main

# https://kr.investing.com/news/cryptocurrency-news

In [ ]:
web_page = 'https://kr.investing.com/news/cryptocurrency-news'
results = []

for i in range(2, 2001):
    html = requests.get(web_page).text
    soup = BeautifulSoup(html, 'html.parser')
    url_tag_list = soup.find_all('article', {"data-test": "article-item"})

    try:
        for url_tag in url_tag_list:
            data = investing(url_tag)
            results.append(data)
    except Exception as e:
        print(e)

    time.sleep(random.uniform(0.25, 1))
    web_page = f'https://kr.investing.com/news/cryptocurrency-news/{i}'

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'


In [21]:
web_page = 'https://kr.investing.com/news/cryptocurrency-news'
results = []

for i in range(2, 2001):
    try:
        html = requests.get(web_page).text
        soup = BeautifulSoup(html, 'html.parser')
        url_tag_list = soup.find_all('article', {"data-test": "article-item"})

        executor = futures.ThreadPoolExecutor(max_workers=len(url_tag_list))
        data = executor.map(investing, url_tag_list)

        for d in data:
            results.append(d)

        time.sleep(random.uniform(0.25, 1))
        web_page = f'https://kr.investing.com/news/cryptocurrency-news/{i}'
    except Exception as e:
        print(e)

In [ ]:
web_page = 'https://kr.investing.com/news/cryptocurrency-news'
results = []

for i in range(2, 5):
    html = requests.get(web_page).text
    soup = BeautifulSoup(html, 'html.parser')
    url_tag_list = soup.find_all('article', {"data-test": "article-item"})

    with futures.ThreadPoolExecutor(max_workers=len(url_tag_list)) as executor:
        future_to_url = {executor.submit(investing, url): url for url in url_tag_list}
        for future in futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                results.append(data)
            except Exception as exc:
                print('%r generated an exception: %s' % (url, exc))
            else:
                print('%r page is %d bytes' % (url, len(data)))

    time.sleep(random.uniform(0.5, 1.75))
    web_page = f'https://kr.investing.com/news/cryptocurrency-news/{i}'

<article class="news-analysis-v2_article__wW0pT flex w-full sm:flex-row-reverse md:flex-row" data-test="article-item"><figure class="hidden justify-center sm:ml-6 sm:block md:ml-0 md:mr-6 h-[59px] w-[80px] md:h-[104px] md:w-[140px]" tabindex="-1"><div class="relative"><div class="absolute z-1 flex items-center justify-center bg-[#8488924d] h-[59px] w-[80px] md:h-[104px] md:w-[140px]"><div class="lazyload-wrapper"><div class="lazyload-placeholder"></div></div></div><div class="lazyload-wrapper"><div class="lazyload-placeholder"></div></div></div></figure><div class="news-analysis-v2_content__z0iLP w-full text-xs sm:flex-1"><a class="text-inv-blue-500 hover:text-inv-blue-500 hover:underline focus:text-inv-blue-500 focus:underline whitespace-normal text-sm font-bold leading-5 !text-[#181C21] sm:text-base sm:leading-6 lg:text-lg lg:leading-7" data-test="article-title-link" href="https://kr.investing.com/news/cryptocurrency-news/article-1293155">비트코인, 사상 첫 1만 달러 돌파</a><p class="mt-[0.5rem] 

In [16]:
web_page = 'https://kr.investing.com/news/cryptocurrency-news'
p0 = re.compile(r'(읽기|provided|[a-z0-9]@[a-z0-9])', flags=re.IGNORECASE)
p1 = re.compile(r'([가-힣])\.(\w)')
p2 = re.compile(r'(\xa0)+')

html = requests.get(web_page, headers=headers).text
soup = BeautifulSoup(html, 'html.parser')
news_list = soup.find_all('article', {"data-test": "article-item"})

for news_url in news_list:
    info = {}
    url = news_url.find("a")["href"]
    html = requests.get(url, headers=headers).text
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find('h1', id='articleTitle').text

    first_upload_time = None
    div = soup.find_all('div', {'class': 'flex flex-row items-center'})
    span = div[1].find('span')
    last_upload_time_list = re.split(r'\s+\r\n\s+', span.text)[1].split()
    y_m_d = '-'.join(times[:-1] for times in last_upload_time_list[:3])
    if last_upload_time_list[3] == '오전':
        ap = 'AM'
    else:
        ap = 'PM'
    last_upload_time = y_m_d + ' ' + ap + ' ' + last_upload_time_list[4]

    author = None

    div = soup.find('div', id='article')
    news_content = div.find_all('p')

    info['news_title'] = title
    info['news_first_upload_time'] = first_upload_time
    info['news_last_upload_time'] = last_upload_time
    info['author'] = author

    if p0.search(news_content[-1].text):
        del news_content[-1]


    text = ''.join(t.text for t in news_content).strip(r' \t\n\r\f\v')
    text = p1.sub(r'\g<1>. \g<2>', text)
    text = p2.sub(' ', text)
    info['news_content'] = text

    info['news_url'] = url
    info['news_website'] = 'Investing'

    print(info)
    break


{'news_title': 'EOS 가격이 14% 상승', 'news_first_upload_time': None, 'news_last_upload_time': '2024-12-06 AM 01:33', 'author': None, 'news_content': 'Investing.com - EOS 가격이 Investing.com Index에서 목요일 01:32 (16:32 GMT)에 13.89% 상승하여 1.4965에 거래되었다. 이는 2024년 12월 3일 이래 최대의 일일 퍼센테이지 상승폭이다. 이 상승으로 EOS의 시가총액이 $2.2578B으로 증가하였다. 이는 전체 암호화폐 시가총액의 0.06%를 차지한다. EOS 시가총액의 이전 최고는 $17.5290B이다. 지난 24시간 동안 EOS 가격은 $1.2687-$1.5145 사이에 거래되었다. 지난 7일간 EOS 가격은 85.88% 상승했다 상승하였다. 지난 24시간동안 EOS의 거래량은 1.0539B이고 전체 암호화폐 거래량의 0.29%를 차지했다. 지난 7일 동안 $0.7950-$1.5350 사이에서 거래되었다. EOS의 현재가격은 2018년 4월 29일의 최고치 $22.98에서 93.49% 하락한 수준이다. 비트코인 가격은 Investing.com Index에서 5.81% 상승한 $101,268.4에 거래되었다. 이더리움 가격은 Investing.com Index에서 1.70% 상승하여 $3,883.88에 거래되었다. 비트코인의 시가총액은 $2,015.6698B로 전체 암호화폐의 54.90%를 차지하고, 이더리움의 시가총액은 $469.3760B로 전체의 12.78%에 달했다.', 'news_url': 'https://kr.investing.com/news/cryptocurrency-news/article-1294240', 'news_website': 'Investing'}


# https://www.hankyung.com/koreamarket/news/crypto

In [ ]:
driver = webdriver.Chrome(options=options)
web_page = 'https://kr.investing.com/news/cryptocurrency-news'
page_num = 1

# 웹 페이지 열기
driver.get(web_page)
elements = driver.find_elements(By.CSS_SELECTOR, "a[data-test='article-title-link']")

In [ ]:
data = []

for ele in elements:
    info = {}
    url = ele.get_attribute("href")
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find('h1', id='articleTitle').text

    start_time = None
    div = soup.find_all('div', {'class': 'flex flex-row items-center'})
    span = div[1].find('span')
    end_time_list = re.split(r'\s+\r\n\s+', span.text)[1].split()
    y_m_d = '-'.join(times[:-1] for times in end_time_list[:3])
    if end_time_list[3] == '오전':
        ap = 'AM'
    else:
        ap = 'PM'
    end_time = y_m_d + ' ' + ap + ' ' + end_time_list[4]

    author = None

    div = soup.find('div', id='article')
    news_content = div.find_all('p')


    info['news_title'] = title
    info['news_upload_start_time'] = start_time
    info['news_upload_end_time'] = end_time
    info['author'] = author

    if '읽기' in news_content[-1].text:
        del news_content[-1]

    info['news_content'] = ''.join(t.text for t in news_content).strip(r' \t\n\r\f\v')

    info['news_url'] = url
    info['news_website'] = 'investing'


    print(title)
    print(end_time)
    print(info['news_content'])
    break

"비트코인, 10만달러 부근서 지지선 형성…알트코인 변동성은 확대 전망"
2024-12-06 AM 04:23
오늘 비트코인(BTC)이 엄청난 상승 랠리로 역대 최고가(ATH)를 경신했지만, 롱 포지션 수요 증가로 펀딩 비율이 과열되면서 시장 변동성이 높아질 수 있다는 전망이 나왔다.5일(현지시간) 가상자산(암호화폐) 전문 미디어 코인데스크는 "비트코인이 광범위한 시장 수요로 인해 10만달러를 넘어섰지만, 강세 레버리지 수요로 인해 시장 과열이 발생했으며, 가상자산 시장에 조정 가능성이 높아지고 있다"라고 전했다.이번 상승세는 도널드 트럼프 대통령 당선인이 가상자산 지지자인 폴 앳킨스를 증권거래위원장(SEC)으로 임명하기로 결정한 이후 발생했다.벨로데이터에 따르면 이번 상승 랠리 이후 비트코인의 연간 자금조달 비율은 약 100%로 치솟았으며, 도지(DOGE), 리플(XRP), 크립토닷컴(CRO), 모네로(XMR) 등 다수 코인들의 자금 조달 비율도 100%를 넘어섰다.펠릭스 하트만 하트만 캐피털 설립자는 "레버리지 사용으로 인해 자금 조달 비율이 높아졌으며, 이로 인해 20~30%의 조정이 발생한다 하더라도 그다지 놀랍지 않을 것"이라고 밝혔다.


In [ ]:
driver = webdriver.Chrome(options=options)
web_page = 'https://www.hankyung.com/koreamarket/news/crypto'
page_num = 1

# 웹 페이지 열기
driver.get(web_page)
elements = driver.find_elements(By.CSS_SELECTOR, "h2.news-tit a")